In [68]:
import concurrent
import datetime
import logging
import urllib
from concurrent.futures import ALL_COMPLETED

import pandas as pd

from nsedt import utils
from nsedt.resources import constants as cns
from nsedt.utils import data_format

log = logging.getLogger("root")


def get_price(
    start_date: datetime,
    end_date: datetime,
    symbol: str,
    response_type: str = "panda_df",
    columns_drop_list: list = None,
    columns_rename_map: map = None,
):
    """
        Get price of index

    Args:

        start_date (datetime): start date
        end_date (datetime): end date
        symbol (str): symbol name or index name
        response_type (str, optional):  Define return type: panda_df or json.
                                        Defaults to "panda_df".
        columns_drop_list (list,optional): define columns drop list, Defaults to None
        columns_rename_map (map, optional): define columns rename map, Defaults to None

    Raises:

        exc: general Exception

    Returns:

            Pandas DataFrame: df containing company info
        or
            Json: json containing company info

    """
    params = {}
    cookies = utils.get_cookies()
    base_url = cns.BASE_URL
    event_api = cns.INDEX_PRICE_HISTORY
    symbol = utils.get_symbol(symbol=symbol, get_key="indices")

    url_list = []

    # set the window size to one year
    window_size = datetime.timedelta(days=cns.WINDOW_SIZE)
    start_date, end_date = utils.check_nd_convert(start_date, end_date)

    current_window_start = start_date
    while current_window_start < end_date:
        current_window_end = current_window_start + window_size

        # check if the current window extends beyond the end_date
        current_window_end = min(current_window_end, end_date)

        params = {
            "indexType": symbol,
            "from": current_window_start.strftime("%d-%m-%Y"),
            "to": current_window_end.strftime("%d-%m-%Y"),
        }
        url = base_url + event_api + urllib.parse.urlencode(params)
        url_list.append(url)

        # move the window start to the next day after the current window end
        current_window_start = current_window_end + datetime.timedelta(days=1)

    result = pd.DataFrame()
    with concurrent.futures.ThreadPoolExecutor(max_workers=cns.MAX_WORKERS) as executor:
        future_to_url = {
            executor.submit(utils.fetch_url, url, cookies, response_type="panda"): url
            for url in url_list
        }
        concurrent.futures.wait(future_to_url, return_when=ALL_COMPLETED)
        for future in concurrent.futures.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                data = future.result()
                if (
                    data.get("data").get("indexCloseOnlineRecords") == []
                    or data.get("data").get("indexTurnoverRecords") == []
                ):
                    continue
                dataframe = data_format.indices(
                    data, columns_drop_list, columns_rename_map
                )
                result = pd.concat([result, dataframe])
            except Exception as exc:
                log.error("%s got exception: %s. Please try again later.", url, exc)
                raise exc

    if response_type == "panda_df":
        return result
    return result.to_json(orient="records")

In [108]:
# for index historical data
from nsedt import indices as ind
from datetime import date

start_date = date(2024, 1, 1)
end_date = date(2024, 4, 10)
#print(ind.get_price(start_date=start_date, end_date=end_date, symbol="NIFTY 50"))
data = ind.get_price(start_date=start_date, end_date=end_date, symbol="NIFTY200 MOMENTUM 30")
# To change date format from '%d-%b-%Y' to '%Y-%m-%d'
data["Date"] = pd.to_datetime(data["Date"],format='%d-%b-%Y')
data

,Open Price,High Price,Close Price,Low Price,Date
0,30336.65,30376.40,29829.25,29727.95,2024-02-21
1,29995.50,30287.60,30231.30,29616.25,2024-02-22
2,30306.45,30433.05,30328.65,30207.30,2024-02-23
3,30324.10,30408.45,30274.75,30153.95,2024-02-26
4,30252.20,30424.50,30256.95,30106.60,2024-02-27
...,...,...,...,...,...
31,29157.75,29928.60,29832.50,29016.35,2024-02-14
32,30033.55,30194.55,30171.15,29917.65,2024-02-15
33,30361.95,30592.05,30515.20,30312.15,2024-02-16
34,30680.75,30776.90,30396.55,30362.50,2024-02-19


In [96]:
#for nse stock historical data
from nsedt import equity as eq
from datetime import date

start_date = date(2023, 1, 1)
end_date = date(2024, 4, 10)
data=eq.get_price(start_date, end_date, symbol="BLUESTARCO")
#start_date = "10-05-2023"
#end_date = "10-04-2024"
#print(eq.get_corpinfo(start_date, end_date, symbol="TCS"))
#print(eq.get_event(start_date, end_date))
#print(eq.get_event())
#print(eq.get_marketstatus())
#print(eq.get_marketstatus(response_type="json"))
#print(eq.get_companyinfo(symbol="TCS"))
#print(eq.get_companyinfo(symbol="TCS", response_type="json"))
#print(eq.get_chartdata(symbol="TCS"))
#print(eq.get_symbols_list()) #print the list of symbols used by NSE for equities
#print(eq.get_asm_list(asm_type = "shortterm"))

In [97]:
data.tail(10)

,Date,Open Price,High Price,Low Price,Close Price,Prev Close Price,Total Traded Quantity,Total Traded Value,52 Week High Price,52 Week Low Price,VWAP,Deliverable Volume,Deliverable Percent,Series
303,2024-03-21,1240.35,1262.20,1237.45,1247.25,1231.60,262805,3.283238e+08,1615.0,701.7,1249.31,199411,75.88,EQ
304,2024-03-22,1247.00,1254.80,1225.00,1241.30,1247.25,222397,2.771783e+08,1615.0,701.7,1246.32,182533,82.08,EQ
305,2024-03-26,1238.70,1272.90,1228.15,1265.45,1241.30,180115,2.268166e+08,1560.0,701.7,1259.29,-,-,EQ
306,2024-03-27,1272.90,1310.60,1262.10,1301.00,1265.45,320622,4.147927e+08,1560.0,701.7,1293.71,209451,65.33,EQ
307,2024-03-28,1319.00,1331.95,1250.00,1273.70,1301.00,479647,6.082550e+08,1560.0,701.7,1268.13,299975,62.54,EQ
308,2024-04-01,1278.00,1301.00,1265.30,1276.90,1273.70,204280,2.616680e+08,1560.0,701.7,1280.93,118514,58.02,EQ
309,2024-04-02,1290.00,1317.35,1282.70,1310.80,1276.90,1158734,1.512428e+09,1560.0,701.7,1305.24,880345,75.97,EQ
310,2024-04-03,1311.40,1365.60,1298.35,1348.25,1310.80,1198119,1.610004e+09,1560.0,701.7,1343.78,746359,62.29,EQ
311,2024-04-04,1365.60,1380.00,1341.00,1348.30,1348.25,553902,7.510640e+08,1560.0,701.7,1355.95,304460,54.97,EQ
312,2024-04-05,1349.00,1370.00,1321.35,1363.35,1348.30,430660,5.861929e+08,1560.0,701.7,1361.15,277558,64.45,EQ


In [93]:
#Equity code if above not worked
import concurrent
import datetime
import logging
import urllib
from concurrent.futures import ALL_COMPLETED

import pandas as pd

from nsedt import utils
from nsedt.resources import constants as cns
from nsedt.utils import data_format

log = logging.getLogger("root")


def get_companyinfo(
    symbol: str,
    response_type: str = "panda_df",
):
    """get_companyinfo

    Args:\n
        symbol (str): stock name\n
        response_type (str, Optional): define the response type panda_df | json. Default panda_df\n
    Returns:\n
        Pandas DataFrame: df containing company info\n
      or\n
        Json: json containing company info\n
    """

    params = {}
    cookies = utils.get_cookies()
    base_url = cns.BASE_URL
    event_api = cns.EQUITY_INFO

    params["symbol"] = symbol

    url = base_url + event_api + urllib.parse.urlencode(params)
    data = utils.fetch_url(
        url,
        cookies,
        key=None,
        response_type=response_type,
    )

    return data


def get_marketstatus(
    response_type: str = "panda_df",
):
    """
    Args:\n
        response_type (str, Optional): define the response type panda_df | json. Default panda_df\n
    Returns:\n
        Pandas DataFrame: df containing market status\n
        Json : Json containing market status\n
    """

    cookies = utils.get_cookies()
    base_url = cns.BASE_URL
    event_api = cns.MARKETSTATUS

    url = base_url + event_api
    data = utils.fetch_url(
        url,
        cookies,
        key="marketState",
        response_type=response_type,
    )

    return data


def get_price(
    start_date,
    end_date,
    symbol=None,
    input_type="stock",
    series="EQ",
):
    """
    Create threads for different requests, parses data, combines them and returns dataframe\n
    Args:\n
        start_date (datetime.datetime): start date\n
        end_date (datetime.datetime): end date\n
        input_type (str): Either 'stock' or 'index'\n
        symbol (str, optional): stock symbol. Defaults to None. TODO: implement for index`\n
    Returns:\n
        Pandas DataFrame: df containing data for symbol of provided date range\n
    """
    cookies = utils.get_cookies()
    base_url = cns.BASE_URL
    price_api = cns.EQUITY_PRICE_HISTORY
    url_list = []

    # set the window size to one year
    window_size = datetime.timedelta(days=cns.WINDOW_SIZE)

    start_date, end_date = utils.check_nd_convert(start_date, end_date)

    current_window_start = start_date
    while current_window_start < end_date:
        current_window_end = current_window_start + window_size

        # check if the current window extends beyond the end_date
        current_window_end = min(current_window_end, end_date)

        if input_type == "stock":
            params = {
                "symbol": symbol,
                "from": current_window_start.strftime("%d-%m-%Y"),
                "to": current_window_end.strftime("%d-%m-%Y"),
                "dataType": "priceVolumeDeliverable",
                "series": series,
            }
            url = base_url + price_api + urllib.parse.urlencode(params)
            url_list.append(url)

        # move the window start to the next day after the current window end
        current_window_start = current_window_end + datetime.timedelta(days=1)

    result = pd.DataFrame()
    with concurrent.futures.ThreadPoolExecutor(max_workers=cns.MAX_WORKERS) as executor:
        future_to_url = {
            executor.submit(utils.fetch_url, url, cookies, "data"): url
            for url in url_list
        }
        concurrent.futures.wait(future_to_url, return_when=ALL_COMPLETED)
        for future in concurrent.futures.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                dataframe = future.result()
                result = pd.concat([result, dataframe])
            except Exception as exc:
                logging.error("%s got exception: %s. Please try again later.", url, exc)
                raise exc
    return data_format.price(result)


def get_corpinfo(
    start_date,
    end_date,
    symbol=None,
    response_type="panda_df",
):
    """
    Create threads for different requests, parses data, combines them and returns dataframe\n
    Args:\n
        start_date (datetime.datetime): start date\n
        end_date (datetime.datetime): end date\n
        symbol (str, optional): stock symbol. Defaults to None.\n
    Returns:\n
        Pandas DataFrame: df containing data for symbol of provided date range\n
      or\n
        Json: json containing data for symbol of provided date range\n
    """
    cookies = utils.get_cookies()
    params = {
        "symbol": symbol,
        "from_date": start_date,
        "to_date": end_date,
        "index": "equities",
    }
    base_url = cns.BASE_URL
    price_api = cns.EQUITY_CORPINFO
    url = base_url + price_api + urllib.parse.urlencode(params)

    data = utils.fetch_url(
        url,
        cookies,
        key=None,
        response_type=response_type,
    )

    return data


def get_event(
    start_date=None,
    end_date=None,
    index="equities",
):
    """
    Args:\n
        start_date (datetime.datetime,optional): start date\n
        end_date (datetime.datetime,optional): end date\n
    Returns:\n
        Pandas DataFrame: df containing event of provided date range\n
    """
    params = {}
    cookies = utils.get_cookies()
    base_url = cns.BASE_URL
    event_api = cns.EQUITY_EVENT

    params["index"] = index
    if start_date is not None:
        params["from_date"] = start_date
    if end_date is not None:
        params["to_date"] = end_date

    url = base_url + event_api + urllib.parse.urlencode(params)
    return utils.fetch_url(url, cookies)


def get_chartdata(
    symbol,
    preopen=False,
    response_type="panda_df",
):
    """
    Args:\n
        symbol (str): stock symbol.\n
    Returns:\n
        Pandas DataFrame: df containing chart data of provided date\n
    """
    params = {}
    cookies = utils.get_cookies()
    base_url = cns.BASE_URL
    event_api = cns.EQUITY_CHART
    try:
        identifier = get_companyinfo(
            symbol,
            response_type="json",
        )[
            "info"
        ]["identifier"]

    except KeyError:
        return f"Invalid symbol name: {symbol}"

    params["index"] = identifier
    if preopen:
        params["preopen"] = "true"

    url = base_url + event_api + urllib.parse.urlencode(params)

    data = utils.fetch_url(
        url,
        cookies,
        key="grapthData",
        response_type=response_type,
    )
    if response_type == "panda_df":
        data_frame = data.rename(
            columns={
                0: "timestamp_milliseconds",
                1: "price",
            }
        )
        data_frame["datetime"] = pd.to_datetime(
            data_frame["timestamp_milliseconds"], unit="ms"
        )
        return data_frame
    return data


def get_symbols_list():
    """
    Args:\n
        No arguments needed\n
    Returns:\n
        List of stock or equity symbols\n
    """
    cookies = utils.get_cookies()
    base_url = cns.BASE_URL
    event_api = cns.EQUITY_LIST

    url = base_url + event_api
    data = utils.fetch_url(url, cookies)
    f_dict = data.to_dict()
    eq_list = []
    for i in range(len(f_dict["data"])):
        eq_list.append(f_dict["data"][i]["metadata"]["symbol"])

    return eq_list


def get_asm_list(asm_type="both") -> list:
    """
        Args:\n
            asm_type (str): ASM type, possible values: both,longterm,shortterm\n
        Returns:\n
            List of stocks under ASM\n
    """
    cookies = utils.get_cookies()
    base_url = cns.BASE_URL
    event_api = cns.ASM_LIST

    url = base_url + event_api
    data = utils.fetch_url(url, cookies)
    _data = data.to_dict()

    if asm_type ==  "both":
        return _data
    if asm_type == "longterm":
        return _data.get("longterm").get("data")
    if asm_type == "shortterm":
        return _data.get("shortterm").get("data")
    return ["possible values are both,longterm,shortterm"]

In [94]:
from nsedt import equity as eq
from datetime import date

start_date = date(2023, 1, 1)
end_date = date(2024, 4, 10)
data=eq.get_price(start_date, end_date, symbol="BLUESTARCO")

In [95]:
data

,Date,Open Price,High Price,Low Price,Close Price,Prev Close Price,Total Traded Quantity,Total Traded Value,52 Week High Price,52 Week Low Price,VWAP,Deliverable Volume,Deliverable Percent,Series
0,2023-01-02,1198.5,1200.65,1180.40,1193.95,1199.55,34247,4.073741e+07,1287.7,836.35,1189.52,16935,49.45,EQ
1,2023-01-03,1199.8,1218.05,1189.05,1204.70,1193.95,276456,3.322600e+08,1287.7,836.35,1201.85,226755,82.02,EQ
2,2023-01-04,1204.1,1207.90,1180.00,1183.75,1204.70,21073,2.516923e+07,1287.7,836.35,1194.38,8294,39.36,EQ
3,2023-01-05,1200.0,1208.95,1182.05,1200.30,1183.75,88787,1.063943e+08,1287.7,836.35,1198.31,73447,82.72,EQ
4,2023-01-06,1190.0,1207.00,1190.00,1202.65,1200.30,26313,3.155786e+07,1287.7,836.35,1199.33,14570,55.37,EQ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,2024-04-01,1278.0,1301.00,1265.30,1276.90,1273.70,204280,2.616680e+08,1560.0,701.70,1280.93,118514,58.02,EQ
309,2024-04-02,1290.0,1317.35,1282.70,1310.80,1276.90,1158734,1.512428e+09,1560.0,701.70,1305.24,880345,75.97,EQ
310,2024-04-03,1311.4,1365.60,1298.35,1348.25,1310.80,1198119,1.610004e+09,1560.0,701.70,1343.78,746359,62.29,EQ
311,2024-04-04,1365.6,1380.00,1341.00,1348.30,1348.25,553902,7.510640e+08,1560.0,701.70,1355.95,304460,54.97,EQ
